In [ ]:
#to aquire full access to the data, connect the google drive folder to this Notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /root/.cache/huggingface/transformers

In [ ]:
!pip install -U --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0

In [ ]:
!pip install transformers==4.36.2
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install monai

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install -U --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0

In [ ]:
import numpy as np
from datasets import Dataset as HFDataset
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import SamProcessor, SamModel
import monai
from tqdm import tqdm
from statistics import mean
import pandas as pd
import os
import random
from scipy import ndimage
import time

In [ ]:
# upload the prepared .npy-files

# train dataset
train_images = np.load("/content/drive/MyDrive/Colab Notebooks/Bachelor/patched_images/train_images_filtered_blacked.npy")
train_masks = np.load("/content/drive/MyDrive/Colab Notebooks/Bachelor/patched_images/train_masks_filtered_blacked.npy")

print("Training data loaded:", train_images.shape, train_masks.shape)

# validation dataset
val_images = np.load("/content/drive/MyDrive/Colab Notebooks/Bachelor/patched_images/val_images_filtered_blacked.npy")
val_masks = np.load("/content/drive/MyDrive/Colab Notebooks/Bachelor/patched_images/val_masks_filtered_blacked.npy")

print("Validation data loaded:", val_images.shape, val_masks.shape)

# In HuggingFace Dataset umwandeln
train_dataset = HFDataset.from_dict({
    "image": [Image.fromarray(img) for img in train_images],
    "label": [Image.fromarray(mask) for mask in train_masks],
})
val_dataset = HFDataset.from_dict({
    "image": [Image.fromarray(img) for img in val_images],
    "label": [Image.fromarray(mask) for mask in val_masks],
})

train_dataset
val_dataset

In [ ]:
import random
import matplotlib.pyplot as plt

img_num = random.randint(0, train_images.shape[0]-1)
example_image = train_dataset[img_num]["image"]
example_mask = train_dataset[img_num]["label"]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first image on the left
axes[0].imshow(np.array(example_image), cmap='gray')  # Assuming the first image is grayscale
axes[0].set_title("Image")

# Plot the second image on the right
axes[1].imshow(example_mask, cmap='gray')  # Assuming the second image is grayscale
axes[1].set_title("Mask")

# Hide axis ticks and labels
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the images side by side
plt.show()

[ ]

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def bbox_from_mask(mask: np.ndarray):
    """Berechnet eine achsenparallele Box aus einer Binärmaske."""
    m = np.asarray(mask)
    if m.ndim == 3:
        m = m[..., 0]
    if m.dtype != np.uint8:
        m = m.astype(np.uint8)
    if m.max() > 1:
        m = (m > 127).astype(np.uint8)

    ys, xs = np.where(m > 0)
    if xs.size == 0:
        return None
    x0, x1 = int(xs.min()), int(xs.max())
    y0, y1 = int(ys.min()), int(ys.max())
    return x0, y0, x1, y1

def _ensure_xyxy(bbox):
    """Saniert Eingabe und korrigiert vertauschte Koordinaten."""
    arr = np.asarray(bbox, dtype=float).reshape(-1)
    if arr.size != 4:
        raise ValueError(f"bbox erwartet vier Werte, erhalten: {arr}")
    x0, y0, x1, y1 = arr.tolist()
    if x1 < x0:
        x0, x1 = x1, x0
    if y1 < y0:
        y0, y1 = y1, y0
    if x1 == x0:
        x1 = x0 + 1.0
    if y1 == y0:
        y1 = y0 + 1.0
    return x0, y0, x1, y1

def jitter_bbox(bbox, H, W,
                trans_px=10,
                scale_min=0.9,
                scale_max=1.1,
                min_size=1):
    """
    Jittert eine Box durch Translation und Skalierung und beschneidet sicher auf das Bild.
    H und W sind Bildhoehe und Bildbreite.
    Rueckgabeformat: np.array([x0, y0, x1, y1], dtype=np.int64)
    """
    x0, y0, x1, y1 = _ensure_xyxy(bbox)
    w = max(x1 - x0, 1.0)
    h = max(y1 - y0, 1.0)
    cx = x0 + 0.5 * w
    cy = y0 + 0.5 * h

    s  = float(np.random.uniform(scale_min, scale_max))
    tx = float(np.random.randint(-trans_px, trans_px + 1)) if trans_px > 0 else 0.0
    ty = float(np.random.randint(-trans_px, trans_px + 1)) if trans_px > 0 else 0.0

    new_w = max(w * s, float(min_size))
    new_h = max(h * s, float(min_size))

    nx0 = int(round(cx - 0.5 * new_w + tx))
    ny0 = int(round(cy - 0.5 * new_h + ty))
    nx1 = nx0 + int(round(new_w))
    ny1 = ny0 + int(round(new_h))

    nx0 = max(0, min(nx0, W - 2))
    ny0 = max(0, min(ny0, H - 2))
    nx1 = max(nx0 + 1, min(nx1, W - 1))
    ny1 = max(ny0 + 1, min(ny1, H - 1))

    return np.array([nx0, ny0, nx1, ny1], dtype=np.int64)

# Zufälligen Index wählen
idx = random.randrange(len(train_dataset))
example_image = np.array(train_dataset[idx]["image"])
example_mask  = np.array(train_dataset[idx]["label"])

# Boxen bestimmen
bbox = bbox_from_mask(example_mask)
H, W = example_mask.shape[:2]
use_jitter = True  # auf False setzen, falls nur die Originalbox gezeigt werden soll
bbox_j = jitter_bbox(bbox, H, W) if (bbox is not None and use_jitter) else None

# Darstellung
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Bild links
axes[0].imshow(example_image, cmap="gray")
axes[0].set_title("Bild")
axes[0].set_xticks([]); axes[0].set_yticks([])

# Maske rechts
axes[1].imshow(example_mask, cmap="gray")
axes[1].set_title("Maske")
axes[1].set_xticks([]); axes[1].set_yticks([])

# Boxen einzeichnen
if bbox is not None:
    x0, y0, x1, y1 = bbox
    w0, h0 = x1 - x0, y1 - y0
    for ax in axes:
        ax.add_patch(Rectangle((x0, y0), w0, h0,
                               fill=False, linewidth=2, edgecolor="lime"))
if bbox_j is not None:
    x0, y0, x1, y1 = bbox_j
    wj, hj = x1 - x0, y1 - y0
    for ax in axes:
        ax.add_patch(Rectangle((x0, y0), wj, hj,
                               fill=False, linewidth=2, edgecolor="orange"))

if bbox is None:
    fig.suptitle("Keine Objektmaske im gewählten Patch")

plt.tight_layout()
plt.show()

In [ ]:
def _ensure_xyxy(bbox):
    arr = np.asarray(bbox, dtype=float).reshape(-1)
    if arr.size != 4:
        raise ValueError(f"bbox erwartet vier Werte, erhalten: {arr}")
    x0, y0, x1, y1 = arr.tolist()
    if x1 < x0:
        x0, x1 = x1, x0
    if y1 < y0:
        y0, y1 = y1, y0
    if x1 == x0:
        x1 = x0 + 1.0
    if y1 == y0:
        y1 = y0 + 1.0
    return x0, y0, x1, y1

def get_bounding_box(mask, pad=20):
    y, x = np.where(mask > 0)
    H, W = mask.shape
    if x.size == 0:
        # Vollbild als Fallback
        x0, y0, x1, y1 = 0, 0, W - 1, H - 1
    else:
        x_min, x_max = int(x.min()), int(x.max())
        y_min, y_max = int(y.min()), int(y.max())
        # zufällige Erweiterung
        x0 = max(0, x_min - np.random.randint(0, pad + 1))
        y0 = max(0, y_min - np.random.randint(0, pad + 1))
        x1 = min(W - 1, x_max + np.random.randint(0, pad + 1))
        y1 = min(H - 1, y_max + np.random.randint(0, pad + 1))
        x0, y0, x1, y1 = _ensure_xyxy([x0, y0, x1, y1])

    # harte Klippen und Mindestbreite sowie Mindesthöhe sichern
    x0 = max(0, min(int(x0), W - 2))
    y0 = max(0, min(int(y0), H - 2))
    x1 = max(x0 + 1, min(int(x1), W - 1))
    y1 = max(y0 + 1, min(int(y1), H - 1))
    return [x0, y0, x1, y1]

def jitter_bbox(bbox, H, W, trans_px=10, scale_min=0.9, scale_max=1.1, min_size=1):
    x0, y0, x1, y1 = _ensure_xyxy(bbox)
    w = max(x1 - x0, 1.0)
    h = max(y1 - y0, 1.0)
    cx = x0 + 0.5 * w
    cy = y0 + 0.5 * h

    s  = float(np.random.uniform(scale_min, scale_max))
    tx = float(np.random.randint(-trans_px, trans_px + 1)) if trans_px > 0 else 0.0
    ty = float(np.random.randint(-trans_px, trans_px + 1)) if trans_px > 0 else 0.0

    new_w = max(w * s, float(min_size))
    new_h = max(h * s, float(min_size))

    nx0 = int(round(cx - 0.5 * new_w + tx))
    ny0 = int(round(cy - 0.5 * new_h + ty))
    nx1 = nx0 + int(round(new_w))
    ny1 = ny0 + int(round(new_h))

    nx0 = max(0, min(nx0, W - 2))
    ny0 = max(0, min(ny0, H - 2))
    nx1 = max(nx0 + 1, min(nx1, W - 1))
    ny1 = max(ny0 + 1, min(ny1, H - 1))

    return np.array([nx0, ny0, nx1, ny1], dtype=np.int64)

# ---------- Datensatz ----------

class SAMDatasetV2(Dataset):
    def __init__(self, dataset, processor,
                 use_jitter=True, trans_px=40, scale_min=0.7, scale_max=1.4, pad=20):
        if isinstance(dataset, SAMDatasetV2):
            dataset = dataset.dataset
        self.dataset    = dataset          # erwartet Felder 'image' und 'label'
        self.processor  = processor        # SamProcessor aus transformers
        self.use_jitter = use_jitter
        self.trans_px   = trans_px
        self.scale_min  = scale_min
        self.scale_max  = scale_max
        self.pad        = pad

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item  = self.dataset[idx]
        image = item["image"]                      # PIL oder np.ndarray
        mask  = np.array(item["label"]).astype(np.uint8)

        H, W = mask.shape
        base_bbox = get_bounding_box(mask, pad=self.pad)

        # robustes Jittering mit Fallback
        try:
            bbox_np = jitter_bbox(
                base_bbox, H, W,
                trans_px=self.trans_px,
                scale_min=self.scale_min,
                scale_max=self.scale_max
            ) if self.use_jitter else np.array(base_bbox, dtype=np.int64)
        except Exception:
            x0, y0, x1, y1 = _ensure_xyxy(base_bbox)
            x0 = max(0, min(int(x0), W - 2))
            y0 = max(0, min(int(y0), H - 2))
            x1 = max(x0 + 1, min(int(x1), W - 1))
            y1 = max(y0 + 1, min(int(y1), H - 1))
            bbox_np = np.array([x0, y0, x1, y1], dtype=np.int64)

        # durch den Processor schicken, damit Boxen zur Modellauflösung transformiert werden
        enc = self.processor(
            images=image,
            input_boxes=[[bbox_np.tolist()]],   # Form [batch, num_boxes, 4]
            return_tensors="pt"
        )

        pixel_values = enc["pixel_values"].squeeze(0)   # [3, Hs, Ws]

        # der Processor liefert input_boxes je nach Version als Liste oder Tensor
        ib = enc["input_boxes"]
        if isinstance(ib, list):
            input_boxes = ib[0]                         # Tensor [num_boxes, 4]
        else:
            input_boxes = ib.squeeze(0)                 # [num_boxes, 4]
        # für das Modell erwarten wir [1, 4]
        if input_boxes.ndim == 1:
            input_boxes = input_boxes.unsqueeze(0)

        sample = {
            "pixel_values": pixel_values,                             # Tensor [3, Hs, Ws]
            "input_boxes":  input_boxes.to(torch.float32),            # Tensor [1, 4]
            "ground_truth_mask": torch.as_tensor(mask, dtype=torch.float32)
        }
        return sample

In [ ]:
# from torch.utils.data import DataLoader # If DataLoader fails to load in the top, it should get importet here

# CUDA-Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🔌 Nutzt Gerät:", device)

# modell and processor
processor = SamProcessor.from_pretrained("facebook/sam-vit-base") # sam_vit_large/huge kcould improve the result
model = SamModel.from_pretrained("facebook/sam-vit-base")

# freeze Encoder
for name, param in model.named_parameters(): # ensures that only gradients for the mask decoder are calculated
    if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
        param.requires_grad_(False)

# push data into gpu
model.to(device) # attaches cuda-device to the model - model switches du CPU constantly otherwise

In [ ]:
from torch.utils.data import DataLoader

# Basis HF Datensätze beibehalten und nicht überschreiben
train_hf = train_dataset if not isinstance(train_dataset, SAMDatasetV2) else train_dataset.dataset
val_hf   = val_dataset   if not isinstance(val_dataset,   SAMDatasetV2) else val_dataset.dataset

# Wrapper einmalig anlegen
train_sam = SAMDatasetV2(train_hf, processor, use_jitter=True)
val_sam   = SAMDatasetV2(val_hf,   processor, use_jitter=True)

# Zunächst ohne Nebenprozesse, damit keine alte Klassenversion in Subprozessen aktiv bleibt
train_loader = DataLoader(train_sam, batch_size=2, shuffle=True,
                          num_workers=0, pin_memory=True, persistent_workers=False)
val_loader   = DataLoader(val_sam,   batch_size=2, shuffle=False,
                          num_workers=0, pin_memory=True, persistent_workers=False)

In [ ]:
# optimizer and loss
# Note to self: Hyperparameter tuning to improve performance here
optimizer = torch.optim.Adam(model.mask_decoder.parameters(), lr=1e-4)
loss_fn = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean') #uses DiceLoss as the loss function

def dice_coeff(pred, gt, eps=1e-4):
    pred = torch.sigmoid(pred)
    if pred.ndim == 3:  # [B,H,W] → [B,1,H,W]
        pred = pred.unsqueeze(1)
    if gt.ndim == 3:
        gt = gt.unsqueeze(1)
    inter = (pred * gt).sum(dim=(1, 2, 3))
    union = pred.sum(dim=(1, 2, 3)) + gt.sum(dim=(1, 2, 3))
    return ((2 * inter + eps) / (union + eps)).mean().item()

def iou(pred, gt, eps=1e-4):
    pred = (torch.sigmoid(pred) > 0.5).float()
    if pred.ndim == 3:
        pred = pred.unsqueeze(1)
    if gt.ndim == 3:
        gt = gt.unsqueeze(1)
    inter = (pred * gt).sum(dim=(1, 2, 3))
    union = (pred + gt).sum(dim=(1, 2, 3)) - inter
    return ((inter + eps) / (union + eps)).mean().item()

# --- 6) History Container ---
history = {
    "epoch":  [],
    "train_loss": [],
    "val_loss":   [],
    "val_dice":   [],
    "val_iou":    [],
    "train_lr":   [],
    "vepoch_time_sec":     []
}

In [ ]:
from statistics import mean

# Gewichte der Epoche mit dem höchsten Dice-Wert als Model speichern
best_val_dice = 0.0

# Training
num_epochs = 20

device = "cuda" if torch.cuda.is_available() else "cpu" # um sicher zu gehen
model.to(device)

for epoch in range(1, num_epochs+1):

    t0 = time.perf_counter()

    # Training
    model.train()

    train_losses = []
    for batch in tqdm(train_loader):
        pixel_values = batch["pixel_values"].to(device)
        input_boxes = batch["input_boxes"].to(device)
        masks = batch["ground_truth_mask"].float().to(device)
        #forward pass
        outputs = model(pixel_values=pixel_values,
                        input_boxes=input_boxes,
                        multimask_output=False)
        # compute loss
        pred_masks = outputs.pred_masks.squeeze(1)
        loss = loss_fn(pred_masks, masks.unsqueeze(1))

        # backward pass (compute gradients of parameters w.r.t. loss)
        optimizer.zero_grad()
        loss.backward()
        # optimize
        optimizer.step()
        train_losses.append(loss.item())

    # Validation

    model.eval()

    val_losses, val_dice_scores, val_iou_scores = [], [], []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            pixel_values = batch["pixel_values"].to(device)
            input_boxes = batch["input_boxes"].to(device)
            masks = batch["ground_truth_mask"].float().to(device)

            outputs = model(pixel_values=pixel_values,
                            input_boxes=input_boxes,
                            multimask_output=False)

            pred_masks = outputs.pred_masks.squeeze(1)

            val_losses.append(loss_fn(pred_masks, masks.unsqueeze(1)).item())
            val_dice_scores.append(dice_coeff(pred_masks, masks.unsqueeze(1)))
            val_iou_scores.append(iou(pred_masks, masks.unsqueeze(1)))

    lr_now = optimizer.param_groups[0]['lr']
    if device == "cuda":
      torch.cuda.synchronize()
    t_epoch = time.perf_counter() - t0

    history["epoch"].append(epoch)
    history["train_loss"].append(mean(train_losses))
    history["val_loss"].append(mean(val_losses))
    history["val_dice"].append(mean(val_dice_scores))
    history["val_iou"].append(mean(val_iou_scores))
    history["train_lr"].append(lr_now)
    history["vepoch_time_sec"].append(t_epoch)

     # Modell speichern, falls val_dice verbessert wurde
    if history["val_dice"][-1] > best_val_dice:
        best_val_dice = history["val_dice"][-1]
        torch.save(model.state_dict(), "best_model.pt")# Modell speichern
        best_model_save_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor/models/best_model_boxpromt_jitter_20ep_1e4.pt"
        torch.save(model.state_dict(), best_model_save_path)

    print(
      f"Ep {epoch} – Train L:{history['train_loss'][-1]:.4f} | "
      f"Val L:{history['val_loss'][-1]:.4f} | "
      f"Dice:{history['val_dice'][-1]:.4f} | "
      f"IoU:{history['val_iou'][-1]:.4f}"
      )

# save History as a Dataframe
df_history = pd.DataFrame(history)

save_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_07_09/1e4/training_history_original.csv"

df_history.to_csv(save_path, index=False)

print(f"✅ Training-History gespeichert unter: {save_path}")


In [ ]:
# save History as a Dataframe
df_history = pd.DataFrame(history)

save_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_05_09/training_history_original_1e4.csv"

df_history.to_csv(save_path, index=False)

print(f"✅ Training-History gespeichert unter: {save_path}")

# Save the trained weights
save_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor/models/sam_finetuned_jitterBoxes_DiceCE_20ep_lr_1e4.pth"
torch.save(model.state_dict(), save_path)
print("✅ Modell gespeichert unter:", save_path)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from IPython.display import display

# Sicherstellen, dass der Zielordner existiert
save_dir = "/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_07_09/1e-4"
os.makedirs(save_dir, exist_ok=True)

# Anzeige des DataFrames (falls noch nicht geladen: z. B. df_combined = pd.read_csv(...))
display(df_history)

# Plot: Loss-Verlauf
plt.figure(figsize=(6, 4))
plt.plot(df_history['train_loss'], marker='o', label='Train Loss')
plt.plot(df_history['val_loss'], marker='o', label='Val Loss')
plt.title('Loss-Verlauf')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
loss_plot_path = os.path.join(save_dir, "loss_plot_jitter.png")
plt.savefig(loss_plot_path, dpi=300, bbox_inches='tight')
plt.show()

# Plot: Validationsmetriken
plt.figure(figsize=(6, 4))
plt.plot(df_history['val_dice'], marker='o', label='Val Dice')
plt.plot(df_history['val_iou'], marker='o', label='Val IoU')
plt.title('Validationsmetriken')
plt.xlabel('Epoche')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
metrics_plot_path = os.path.join(save_dir, "metrics_plot_jitter.png")
plt.savefig(metrics_plot_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Plots gespeichert unter:\n→ {loss_plot_path}\n→ {metrics_plot_path}")

## Inference


In [ ]:
!pip install patchify

In [ ]:
import os
import glob
import json
import time
import tifffile
import patchify
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional
from PIL import Image
import torch
from transformers import SamModel, SamConfig, SamProcessor

best_model_save_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_07_09/1e-5/best_model_boxpromt_jitter_20ep.pt"

image_folder     = "/content/drive/MyDrive/Colab Notebooks/Bachelor/test_data/images"     # .tif
tps_mask_folder  = "/content/drive/MyDrive/Colab Notebooks/Bachelor/test_data/tps_layer"  # .png (ROI/TPS)
gt_folder        = "/content/drive/MyDrive/Colab Notebooks/Bachelor/test_data/gt_masks"   # .png (GT für HA)
out_folder       = "/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_08_09/Test/SAM"
os.makedirs(out_folder, exist_ok=True)
#out_folder       = "/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_08_09/Test/MY_HA"
#os.makedirs(out_folder, exist_ok=True)

# Masken und Geometrie
def invert_mask(path: str) -> Tuple[np.ndarray, List[np.ndarray]]:
    mask_orig = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if mask_orig is None:
        raise FileNotFoundError(f"Maske nicht gefunden: {path}")
    contour_TPS, _ = cv2.findContours(mask_orig, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.ones_like(mask_orig, dtype=np.uint8) * 255
    cv2.drawContours(mask, contour_TPS, -1, 0, thickness=cv2.FILLED)
    return mask, contour_TPS

def apply_mask(img: np.ndarray, mask: np.ndarray, mode: str = "and") -> np.ndarray:
    if mode == "and": return cv2.bitwise_and(img, img, mask=mask)
    if mode == "or":  return cv2.bitwise_or(img, mask)
    raise ValueError("apply_mask: mode muss 'and' oder 'or' sein")

def shift_mask_binary(m: np.ndarray, dx: int, dy: int) -> np.ndarray:
    h, w = m.shape
    out = np.zeros_like(m)
    # Zielbereich im Ausgabebild
    y0 = max(0,  dy);  y1 = min(h, h + dy)
    x0 = max(0,  dx);  x1 = min(w, w + dx)
    # Quellbereich im Eingabebild
    yy0 = max(0, -dy); yy1 = yy0 + (y1 - y0)
    xx0 = max(0, -dx); xx1 = xx0 + (x1 - x0)
    if y1 > y0 and x1 > x0:
        out[y0:y1, x0:x1] = m[yy0:yy1, xx0:xx1]
    return out

def pad_to_multiple(a: np.ndarray, multiple: int) -> np.ndarray:
    H, W = a.shape[:2]
    padH = (multiple - (H % multiple)) % multiple
    padW = (multiple - (W % multiple)) % multiple
    if padH or padW:
        pads = ((0, padH), (0, padW)) + ((0, 0),) * (a.ndim - 2)
        a = np.pad(a, pads, mode="constant", constant_values=0)
    return a

def ceil_to_multiple(n: int, m: int) -> int:
    return ((n + m - 1) // m) * m

def mask_to_box(mask: np.ndarray, margin: int = 4, min_pixels: int = 1):
    assert mask.ndim == 2
    H, W = mask.shape
    ys, xs = np.where(mask > 0)
    if ys.size < min_pixels:
        return None
    y0, y1 = int(ys.min()), int(ys.max())
    x0, x1 = int(xs.min()), int(xs.max())
    if margin > 0:
        y0 = max(0, y0 - margin)
        x0 = max(0, x0 - margin)
        y1 = min(H - 1, y1 + margin)
        x1 = min(W - 1, x1 + margin)
    return float(x0), float(y0), float(x1), float(y1)


def make_cosine_window(p: int, eps: float = 1e-3) -> np.ndarray:

    w1 = np.hanning(p).astype(np.float32)
    w2 = np.outer(w1, w1).astype(np.float32)
    return np.maximum(w2, eps)

def reconstruct_overlapping(patches: np.ndarray,
                            coords: np.ndarray,
                            H_pad: int,
                            W_pad: int,
                            patch_size: int,
                            window: np.ndarray) -> np.ndarray:

    acc  = np.zeros((H_pad, W_pad), dtype=np.float32)
    wsum = np.zeros((H_pad, W_pad), dtype=np.float32)
    for patch, (y0, x0) in zip(patches, coords):
        y1 = y0 + patch_size
        x1 = x0 + patch_size
        acc[y0:y1, x0:x1]  += patch.astype(np.float32) * window
        wsum[y0:y1, x0:x1] += window
    return acc / np.maximum(wsum, 1e-6)

def reconstruct_overlapping_binary(patches: np.ndarray,
                                   coords: np.ndarray,
                                   H_pad: int,
                                   W_pad: int,
                                   patch_size: int) -> np.ndarray:
    acc = np.zeros((H_pad, W_pad), dtype=np.uint8)
    for patch, (y0, x0) in zip(patches, coords):
        y1 = y0 + patch_size
        x1 = x0 + patch_size
        cur = acc[y0:y1, x0:x1]
        np.maximum(cur, patch, out=cur)
        acc[y0:y1, x0:x1] = cur
    return acc

def thickness_per_column(binary_mask_0_255: np.ndarray) -> np.ndarray:
    return np.sum(binary_mask_0_255 == 255, axis=0).astype(np.int32)

def dice_iou(pred_0_255: np.ndarray, gt_0_255: np.ndarray) -> Tuple[float, float]:
    p = (pred_0_255 > 0).astype(np.uint8)
    g = (gt_0_255   > 0).astype(np.uint8)
    inter = np.sum((p == 1) & (g == 1))
    p_sum = np.sum(p)
    g_sum = np.sum(g)
    union = p_sum + g_sum - inter
    dice = (2.0 * inter) / (p_sum + g_sum) if (p_sum + g_sum) > 0 else 1.0
    iou  = (inter / union) if union > 0 else 1.0
    return float(dice), float(iou)

def coverage_recall(pred_0_255: np.ndarray, gt_0_255: np.ndarray) -> float:
    p = (pred_0_255 > 0)
    g = (gt_0_255   > 0)
    gt_pos = np.sum(g)
    if gt_pos == 0:
        return 1.0 if np.sum(p) == 0 else 0.0
    return float(np.sum(p & g) / gt_pos)

def basename_noext(p: str) -> str:
    return os.path.splitext(os.path.basename(p))[0]

In [ ]:

### SAM TRAINIERT ###

model_config = SamConfig.from_pretrained("facebook/sam-vit-base")
processor    = SamProcessor.from_pretrained("facebook/sam-vit-base")
my_HA_model  = SamModel(config=model_config)
my_HA_model.load_state_dict(torch.load(best_model_save_path))

device = "cuda" if torch.cuda.is_available() else "cpu"
my_HA_model.to(device)
my_HA_model.eval()

'''
### SAM UNTRAINIERT ###
model_name = "facebook/sam-vit-base"
# Processor und Standard SAM laden
processor = SamProcessor.from_pretrained(model_name)
sam_model = SamModel.from_pretrained(model_name)
# Gerät wählen und Modell in den Auswertungsmodus setzen
device = "cuda" if torch.cuda.is_available() else "cpu"
sam_model.to(device)
sam_model.eval()
'''

# PARAMETER

patch_size = 256
step       = 128
final_thr  = 0.80
save_viz   = True
shift_px   = 110          # Verschiebung in Pixel nach oben für HA-Streifen
margin    = 4            # Box-Rand in Pixel (Puffer rund um die eigentliche Box)
min_px    = 1            # Minimale Pixelanzahl für eine Box



# PAIRING
# Bild <-> TPS-Schicht <-> Ground Truth

imgs  = sorted(glob.glob(os.path.join(image_folder, "*.tif")))
tps   = sorted(glob.glob(os.path.join(tps_mask_folder, "*.png")))
gts   = sorted(glob.glob(os.path.join(gt_folder, "*.png")))

img_map = {basename_noext(p): p for p in imgs}
tps_map = {basename_noext(p): p for p in tps}
gt_map  = {basename_noext(p): p for p in gts}

common = sorted(set(img_map) & set(tps_map) & set(gt_map))
pairs = [(img_map[k], tps_map[k], gt_map[k]) for k in common]
print(f"Gefundene Tripel: {len(pairs)}")

# PROCESSING

win = make_cosine_window(patch_size)

for idx, (img_path, tps_path, gt_path) in enumerate(pairs, 1):
    stem = basename_noext(img_path)
    print(f"[{idx}/{len(pairs)}] {stem}")
    t0 = time.perf_counter()
    try:
        # Laden
        large_img = tifffile.imread(img_path)
        if large_img is None or large_img.ndim != 2:
            raise ValueError("Erwarte zweidimensionales Graubild")

        tps_inv, _ = invert_mask(tps_path)            # 255 außerhalb TPS
        img_masked = apply_mask(large_img, tps_inv)   # Bild auf TPS begrenzen

        gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
        if gt is None:
            raise FileNotFoundError(f"GT nicht gefunden: {gt_path}")
        gt_bin = (gt > 0).astype(np.uint8) * 255
        gt_masked = cv2.bitwise_and(gt_bin, tps_inv)  # GT auf TPS begrenzen

        # HA Streifen aus verschobener Nullregion erzeugen
        zero_mask = (img_masked == 0).astype(np.uint8) * 255
        kernel = np.ones((3, 3), np.uint8)
        zero_mask = cv2.morphologyEx(zero_mask, cv2.MORPH_OPEN, kernel)
        m_orig  = (zero_mask > 0).astype(np.uint8)
        m_shift = shift_mask_binary(m_orig, dx=0, dy=-shift_px)
        ha_mask = ((m_shift == 1) & (m_orig == 0)).astype(np.uint8) * 255

        # Patch Bildung
        assert img_masked.shape == gt_masked.shape == ha_mask.shape
        H_orig, W_orig = img_masked.shape
        img_pad = pad_to_multiple(img_masked, patch_size)
        gt_pad  = pad_to_multiple(gt_masked,  patch_size)
        ha_pad  = pad_to_multiple(ha_mask,    patch_size)

        img_p = patchify.patchify(img_pad, (patch_size, patch_size), step=step)
        gt_p  = patchify.patchify(gt_pad,  (patch_size, patch_size), step=step)
        ha_p  = patchify.patchify(ha_pad,  (patch_size, patch_size), step=step)

        img_patches, gt_patches, ha_patches, coords = [], [], [], []
        for i in range(img_p.shape[0]):
            for j in range(img_p.shape[1]):
                y0 = i * step
                x0 = j * step
                img_patches.append(img_p[i, j])
                gt_patches.append(gt_p[i, j].astype(np.uint8))
                ha_patches.append(ha_p[i, j].astype(np.uint8))
                coords.append((y0, x0))
        img_patches = np.array(img_patches)
        gt_patches  = np.array(gt_patches)
        ha_patches  = np.array(ha_patches)
        coords      = np.array(coords, dtype=np.int32)

        # Inferenz mit SAM Standard und HA Box
        N, H, W = img_patches.shape
        probs = np.zeros((N, H, W), dtype=np.float32)

        prompt_log = {
            "promptart": "box",
            "quelle": "ha",
            "parameter": {"margin": margin, "min_pixels": min_px,
                          "patch_size": patch_size, "step": step,
                          "shift_px": shift_px},
            "prompts": []
        }

        with torch.no_grad():
            for k in range(N):
                img_patch = img_patches[k]
                ha_patch  = ha_patches[k]

                box = mask_to_box(ha_patch, margin=margin, min_pixels=min_px)

                # Fallback Box wenn keine Region gefunden wurde
                if box is None:
                    h, w = img_patch.shape
                    cy = max(0, min(h - 1, h // 2))
                    band = 2
                    y0b = max(0, cy - band)
                    y1b = min(h - 1, cy + band)
                    box = [0, y0b, w - 1, y1b]

                x0, y0, x1, y1 = box
                pil_img = Image.fromarray(img_patch).convert("RGB")

                inputs = processor(
                    images=pil_img,
                    input_boxes=[[[x0, y0, x1, y1]]],
                    return_tensors="pt"
                )
                inputs = {kk: vv.to(device) for kk, vv in inputs.items()}

                #out = sam_model(**inputs, multimask_output=False) # Für untrainierten SAM
                out = my_HA_model(**inputs, multimask_output=False) # Für trainierten SAM
                p_map = torch.sigmoid(out.pred_masks[0, 0, 0])  # Form H mal W
                probs[k] = p_map.cpu().numpy()

                gy, gx = int(coords[k][0]), int(coords[k][1])
                box_global = [float(x0 + gx), float(y0 + gy), float(x1 + gx), float(y1 + gy)]
                prompt_log["prompts"].append({
                    "patch_index": int(k),
                    "patch_top_left": [gy, gx],
                    "box_global_xyxy": box_global
                })

        # Rekonstruktion
        H_pad = ceil_to_multiple(H_orig, patch_size)
        W_pad = ceil_to_multiple(W_orig, patch_size)

        full_prob = reconstruct_overlapping(probs, coords, H_pad, W_pad, patch_size, win)
        full_pred = (full_prob > final_thr).astype(np.uint8) * 255
        full_img  = reconstruct_overlapping(img_patches.astype(np.float32), coords, H_pad, W_pad, patch_size, win)
        full_gt   = reconstruct_overlapping_binary(gt_patches, coords, H_pad, W_pad, patch_size)

        # Zuschnitt auf Originalmaß
        full_img_c  = full_img[:H_orig, :W_orig]
        full_prob_c = full_prob[:H_orig, :W_orig]
        full_pred_c = full_pred[:H_orig, :W_orig]
        full_gt_c   = full_gt[:H_orig, :W_orig]

        # Artefakte speichern
        prob_u8 = np.rint(np.clip(full_prob_c * 255.0, 0, 255)).astype(np.uint8)
        prob_dir = os.path.join(out_folder, "prob_maps")
        os.makedirs(prob_dir, exist_ok=True)
        out_prob = os.path.join(prob_dir, f"{stem}_prob.png")

        pred_dir = os.path.join(out_folder, "pred_masks")
        os.makedirs(pred_dir, exist_ok=True)
        out_pred = os.path.join(pred_dir, f"{stem}_pred.png")

        gt_dir = os.path.join(out_folder, "gt_mask")
        os.makedirs(gt_dir, exist_ok=True)
        out_gt = os.path.join(gt_dir, f"{stem}_gt.png")

        cv2.imwrite(out_prob, prob_u8)
        cv2.imwrite(out_pred, full_pred_c)
        cv2.imwrite(out_gt,   full_gt_c)

        # Dickenprofile
        th_pred = thickness_per_column(full_pred_c)
        th_gt   = thickness_per_column(full_gt_c)

        thickness_pred_dir = os.path.join(out_folder, "thickness_pred")
        os.makedirs(thickness_pred_dir, exist_ok=True)
        out_th_pred = os.path.join(thickness_pred_dir, f"{stem}_thickness_pred.csv")

        thickness_gt_dir = os.path.join(out_folder, "thickness_gt")
        os.makedirs(thickness_gt_dir, exist_ok=True)
        out_th_gt   = os.path.join(thickness_gt_dir, f"{stem}_thickness_gt.csv")

        idxs = np.arange(W_orig, dtype=np.int32)
        np.savetxt(out_th_pred, np.c_[idxs, th_pred], delimiter=",",
                   header="spaltenindex,dicke_pixel", fmt="%d", comments="")
        np.savetxt(out_th_gt,   np.c_[idxs, th_gt],   delimiter=",",
                   header="spaltenindex,dicke_pixel", fmt="%d", comments="")

        # Bildmetriken
        dice, iou = dice_iou(full_pred_c, full_gt_c)
        diff = th_pred.astype(np.int32) - th_gt.astype(np.int32)
        mae  = float(np.mean(np.abs(diff))) if diff.size else 0.0
        bias = float(np.mean(diff)) if diff.size else 0.0
        mabs = int(np.max(np.abs(diff))) if diff.size else 0
        cover = coverage_recall(full_pred_c, full_gt_c)
        t_ms = (time.perf_counter() - t0) * 1000.0

        metrics_dir = os.path.join(out_folder, "metrics")
        os.makedirs(metrics_dir, exist_ok=True)

        metrics = {
            "dice": dice,
            "iou": iou,
            "mae_dicke": mae,
            "bias_dicke": bias,
            "max_abs_err": mabs,
            "abdeckung": cover,
            "laufzeit_ms": float(round(t_ms, 1))
        }
        with open(os.path.join(metrics_dir, f"{stem}_metrics.json"), "w") as f:
            json.dump(metrics, f, indent=2)

        # Prompt Protokoll
        prompts_dir = os.path.join(out_folder, "prompts")
        os.makedirs(prompts_dir, exist_ok=True)
        with open(os.path.join(prompts_dir, f"{stem}_prompt_protokoll.json"), "w") as f:
            json.dump(prompt_log, f, indent=2)

        # Bildmetadaten
        metadata_dir = os.path.join(out_folder, "metadata")
        os.makedirs(metadata_dir, exist_ok=True)
        metadata = {
            "bildkennung": stem,
            "patchgroesse": int(patch_size),
            "schrittweite": int(step),
            "ueberlappung": float(step / patch_size),
            "schwellenwert": float(final_thr),
            "verwendete_nachbearbeitung": "keine",
            "prompt_quelle": "ha",
            "ha_shift_px": int(shift_px),
            "blending": "cosine_window_hanning_eps1e-3",
            "prob_map_skalierung": "round(prob*255)"
        }
        with open(os.path.join(metadata_dir, f"{stem}_bildmetadaten.json"), "w") as f:
            json.dump(metadata, f, indent=2)

        # Visualisierung
        if save_viz:
            viz_dir = os.path.join(out_folder, "viz")
            os.makedirs(viz_dir, exist_ok=True)
            fig, axes = plt.subplots(1, 4, figsize=(16, 4))
            axes[0].imshow(full_img_c, cmap="gray"); axes[0].set_title("Bild"); axes[0].axis("off")
            axes[1].imshow(full_gt_c, cmap="gray");  axes[1].set_title("GT"); axes[1].axis("off")
            axes[2].imshow(prob_u8, vmin=0, vmax=255); axes[2].set_title("Wahrscheinlichkeit"); axes[2].axis("off")
            axes[3].imshow(full_pred_c, cmap="gray"); axes[3].set_title("Vorhersage"); axes[3].axis("off")
            plt.tight_layout()
            plt.savefig(os.path.join(viz_dir, f"{stem}_viz.png"), dpi=150)
            plt.close(fig)

        print(f"{stem}: prob, pred, profile und JSON gespeichert")

        if device == "cuda":
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"Fehler bei {stem}: {e}")

In [ ]:
from pathlib import Path
import re

# Wurzelordner anpassen, falls nötig
base_dir = Path("/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_08_09/SAM/gt_mask")

pattern = re.compile(r"_bin\.png$", re.IGNORECASE)

renamed = 0
skipped_exists = 0

candidates = [p for p in base_dir.rglob("*.png") if pattern.search(p.name)]
print(f"Gefundene Kandidaten: {len(candidates)}")

for p in candidates:
    target = p.with_name(pattern.sub("_gt.png", p.name))
    if target.exists():
        print(f"übersprungen, Ziel existiert bereits: {target}")
        skipped_exists += 1
        continue
    p.rename(target)
    renamed += 1

print(f"Umbenannt: {renamed}")
print(f"Übersprungen wegen vorhandenem Ziel: {skipped_exists}")
print("Fertig")

In [ ]:
from pathlib import Path
import re
import math
import numpy as np
import pandas as pd
from scipy import stats

# Pfade
base_dir = Path("/content/drive/MyDrive/Colab Notebooks/Bachelor/Plots/Plots_Von_08_09/Test/SAM")
pred_dir = base_dir / "thickness_pred"
gt_dir   = base_dir / "thickness_gt"

# ---------- Helfer ----------

def has_header(csv_path):
    """Erkennt simpel, ob die erste Zeile eher Header ist."""
    with open(csv_path, 'r', encoding='utf-8', errors='ignore') as f:
        first = f.readline()
    # Wenn Buchstaben in der ersten Zeile stecken, werten wir das als Header
    return bool(re.search(r'[A-Za-zÄÖÜäöü]', first))

def read_thickness_csv(csv_path):
    """Liest eine 2-Spalten-CSV: [spaltenindex, dicke_pixel]. Robust gegen Header und Dezimalkomma."""
    if has_header(csv_path):
        df = pd.read_csv(csv_path)
        # versuche die wahrscheinlichsten Spaltennamen
        cols = list(df.columns)
        if len(cols) < 2:
            # Fallback: ohne Header lesen
            df = pd.read_csv(csv_path, header=None, names=["spaltenindex", "dicke_pixel"])
        else:
            # mappe möglichst auf Standardnamen
            mapping = {}
            # erstbeste zwei Spalten nehmen, aber sinnvoll benennen
            mapping[cols[0]] = "spaltenindex"
            mapping[cols[1]] = "dicke_pixel"
            df = df.rename(columns=mapping)[["spaltenindex", "dicke_pixel"]]
    else:
        df = pd.read_csv(csv_path, header=None, names=["spaltenindex", "dicke_pixel"])

    # Dezimalkomma abfangen und auf float casten
    for c in ["spaltenindex", "dicke_pixel"]:
        ser = df[c].astype(str).str.replace(',', '.', regex=False)
        df[c] = pd.to_numeric(ser, errors='coerce')

    # sortieren, Duplikate im Index vermeiden
    df = df.drop_duplicates(subset=["spaltenindex"]).sort_values("spaltenindex").reset_index(drop=True)
    return df

def clean_stem(p: Path):
    """
    Entfernt Suffixe wie _pred, _gt, _thickness_pred, _thickness_gt
    sowie optionale Dateiendungen, liefert den gemeinsamen Stamm.
    """
    stem = p.stem
    # erst thickness Varianten
    stem = re.sub(r'_(?:thickness_)?(pred|gt)$', '', stem, flags=re.IGNORECASE)
    return stem

def compute_metrics(errors: np.ndarray, total_cols: int):
    """
    Gibt Kennwerte als Dict zurück.
    errors: 1D-Array der validen Fehler e(j) = d_pred - d_ref
    total_cols: Gesamtzahl der Spalten (vor Filter), für Abdeckung
    """
    W = int(np.sum(np.isfinite(errors)))
    if W == 0:
        return dict(W=0, total_columns=int(total_cols), coverage=np.nan,
                    MAE=np.nan, Bias=np.nan, Std=np.nan,
                    CI95_lower=np.nan, CI95_upper=np.nan,
                    MaxAbsError=np.nan)

    e = errors[np.isfinite(errors)]
    mae = float(np.mean(np.abs(e)))
    bias = float(np.mean(e))
    std = float(np.std(e, ddof=1)) if W > 1 else np.nan

    if W > 1 and np.isfinite(std):
        tcrit = stats.t.ppf(0.975, df=W-1)
        half_width = tcrit * std / math.sqrt(W)
        ci_lo = bias - half_width
        ci_hi = bias + half_width
    else:
        ci_lo = np.nan
        ci_hi = np.nan

    max_abs = float(np.max(np.abs(e)))
    coverage = float(W / total_cols) if total_cols > 0 else np.nan

    return dict(W=W, total_columns=int(total_cols), coverage=coverage,
                MAE=mae, Bias=bias, Std=std,
                CI95_lower=ci_lo, CI95_upper=ci_hi,
                MaxAbsError=max_abs)

# ---------- Dateien paaren ----------

pred_files = list(pred_dir.glob("*.csv"))
gt_files   = list(gt_dir.glob("*.csv"))

pred_map = {clean_stem(p): p for p in pred_files}
gt_map   = {clean_stem(p): p for p in gt_files}

common_stems = sorted(set(pred_map.keys()) & set(gt_map.keys()))
missing_pred = sorted(set(gt_map.keys()) - set(pred_map.keys()))
missing_gt   = sorted(set(pred_map.keys()) - set(gt_map.keys()))

if missing_pred:
    print("Warnung: Es fehlen _pred Dateien für:", missing_pred)
if missing_gt:
    print("Warnung: Es fehlen _gt Dateien für:", missing_gt)

# ---------- Metriken je Paar und gesamt ----------

rows = []
all_errors = []

for stem in common_stems:
    df_pred = read_thickness_csv(pred_map[stem]).rename(columns={"dicke_pixel": "d_pred"})
    df_gt   = read_thickness_csv(gt_map[stem]).rename(columns={"dicke_pixel": "d_ref"})

    # Outer-Join auf Spaltenindex, um Gesamtzahl der Spalten zu bestimmen
    merged = pd.merge(df_pred[["spaltenindex", "d_pred"]],
                      df_gt[["spaltenindex", "d_ref"]],
                      on="spaltenindex", how="outer", sort=True)

    # Gesamtzahl vor dem Filter
    total_cols = merged["spaltenindex"].nunique()

    # valide Paare: beide Werte endlich
    valid = merged[["d_pred", "d_ref"]].apply(np.isfinite).all(axis=1)
    merged_valid = merged.loc[valid].copy()

    # Fehler
    merged_valid["e"] = merged_valid["d_pred"] - merged_valid["d_ref"]

    # Kennwerte
    metrics = compute_metrics(merged_valid["e"].to_numpy(), total_cols)
    metrics["datei_stamm"] = stem
    rows.append(metrics)

    # für Gesamt
    all_errors.append(merged_valid["e"].to_numpy())

# Gesamt über alle Paare
if all_errors:
    all_errors_vec = np.concatenate(all_errors)
    total_cols_sum = sum(r["total_columns"] for r in rows)  # Summe der Spalten über alle Dateien
    overall = compute_metrics(all_errors_vec, total_cols_sum)
    overall["datei_stamm"] = "ALLE_DATEIEN"
    rows.append(overall)

# ---------- Ergebnis als Tabelle ----------

cols_order = ["datei_stamm", "W", "total_columns", "coverage",
              "MAE", "Bias", "Std", "CI95_lower", "CI95_upper", "MaxAbsError"]

result_df = pd.DataFrame.from_records(rows)

if result_df.empty:
    print("Es wurden keine Paare ausgewertet. Prüfe die Dateinamen in thickness_pred und thickness_gt.")
    result_df = pd.DataFrame(columns=cols_order)
else:
    # fehlende Spalten ergänzen, Reihenfolge erzwingen
    for c in cols_order:
        if c not in result_df:
            result_df[c] = np.nan
    result_df = result_df[cols_order]

num_cols = ["coverage", "MAE", "Bias", "Std", "CI95_lower", "CI95_upper", "MaxAbsError"]
for c in num_cols:
    if c in result_df:
        result_df[c] = result_df[c].astype(float).round(6)

out_path = base_dir / "thickness_metrics_summary_2.csv"
result_df.to_csv(out_path, index=False)
print(f"Fertig. Zusammenfassung gespeichert unter:\n{out_path}")
display(result_df.head(10))